# Introdcution to views

A **view** is a named query stored in the PostgreSQL database. 
* A view is defined based on one or more tables, which are known as **base tables**.
* The query that defines the view is referred to as a **defining query**.

After creating a view, you can query data from it as you would from a regular table. Behind the scenes, PostgreSQL will rewrite the query against the view and its defining query, executing it to retrieve data from the base tables.

Views do not store data except for the **materialized views**. In PostgreSQL, you can create special views called materialized views that store data physically and periodically refresh it from the base tables.

The materialized views are handy in various scenarios, providing faster data access to a remote server and serving as an effective caching mechanism.

# Advantages of views

**1) Simplifying complex queries**
* Views help simplify complex queries.
* Instead of dealing with joins, aggregations, or filtering conditions, you can query from views as if they were regular tables.
* Typically, first, you create views based on complex queries and store them in the database.
* Then, you can use simple queries based on views instead of using complex queries.

**2) Security and access control**
* Views enable fine-grained control over data access.
* You can create views that expose subsets of data in the base tables, hiding sensitive information.
* This is particularly useful when you have applications that require access to distinct portions of the data.

**3) Logical data independence**
* If your applications use views, you can freely modify the structure of the base tables.
* In other words, views enable you to create a layer of abstraction over the underlying tables.

# CREATE VIEW statement

```postgresql
CREATE VIEW view_name AS
query;
```

**Using the `CREATE VIEW` statement to create a view based on a complex query**
```postgresql
CREATE VIEW customer_info AS
    SELECT first_name, last_name, email, phone, city, postal_code, country
    FROM customer
    INNER JOIN address USING (address_id)
    INNER JOIN city USING (city_id)
    INNER JOIN country USING (country_id);

SELECT * FROM customer_info; 
```

**Creating a view based on another view**
```postgresql
CREATE VIEW customer_usa AS
    SELECT * FROM
    customer_info
    WHERE
    country = 'United States'; 

SELECT * FROM customer_usa; 
```

# Replacing a view

To change the defining query of a view, you use the `CREATE OR REPLACE VIEW` statement:
```postgresql
CREATE OR REPLACE VIEW view_name AS
query;
```

If the view already exists, the statement replaces the existing view; otherwise, it creates a new view.

**Example**:
```postgresql
CREATE OR REPLACE VIEW contact AS
    SELECT first_name, last_name, email, phone
    FROM customer
    INNER JOIN address USING (address_id); 
```

# DROP VIEW statement

**SYNTAX**
```postgresql
DROP VIEW [IF EXISTS] view_name
[CASCADE | RESTRICT]; 
```

* First, specify the name of the view in the `DROP VIEW` clause.
* Second, use `IF EXISTS` to prevent an error if the view does not exist.
    * PostgreSQL will issue a notice instead of an error when you attempt to remove a nonexistent view.
    * The `IF EXISTS` is optional.
* Third, use the `CASCADE` option to remove dependent objects along with the view, or the `RESTRICT` option to reject the removal of the view if other objects depend on the view.
    * The RESTRICT option is the default.
 
**Dropping multiple views**
```postgresql
DROP VIEW [IF EXISTS] view_name1, view_name2, ...
[CASCADE | RESTRICT];
```

**Permissions**: To execute the `DROP VIEW` statement, you need to be the owner of the view or have a `DROP` privilege on it.

**Using the DROP VIEW statement to drop a view that has dependent objects**
```postgresql
DROP VIEW film_info; 
```

PostgreSQL issued an error:
```postgresql
ERROR:  cannot drop view film_info because other objects depend on it
DETAIL:  view horror_film depends on view film_info
HINT:  Use DROP ... CASCADE to drop the dependent objects too. 
```

To drop the view `film_info`, you need to drop its dependent object first or use the `CASCADE` option like this:
```postgresql
DROP VIEW film_info
CASCADE; 
```

This statement drops the `film_info` view as well as its dependent object, which is the horror_film. 

It issued the following notice:
```postgresql
NOTICE:  drop cascades to view horror_film
```

# Updatable Views

A view can be updatable if it meets certain conditions. This means that you can insert, update, or delete data from the underlying tables via the view.

A view is updatable when it meets the following conditions:

**First, the defining query of the view must have exactly one entry in the FROM clause, which can be a table or another updatable view.**

**Second, the defining query must not contain one of the following clauses at the top level:**
* GROUP BY
* HAVING
* LIMIT
* OFFSET FETCH
* DISTINCT
* WITH
* UNION
* INTERSECT
* EXCEPT

**Third, the selection list of the defining query must not contain any:**
* Window functions
* Set-returning function
* Aggregate functions

An updatable view may contain both updatable and non-updatable columns. If you attempt to modify a non-updatable column, PostgreSQL will raise an error.

When you execute a modification statement, such as `INSERT`, `UPDATE`, or `DELETE` to an updatable view, PostgreSQL will convert this statement into the corresponding statement of the underlying table.

If you have a `WHERE` condition in the defining query of a view, you still can update or delete the rows that are not visible through the view. However, if you want to avoid this, you can use the `WITH CHECK OPTION` to define the view.

# WITH CHECK OPTION clause

In PostgreSQL, a view is a named query stored in the PostgreSQL database server. 
* A simple view can be **updatable**.
* To ensure that any data modification made through a view adheres to certain conditions in the view’s definition, you use the `WITH CHECK OPTION` clause.

**SYNTAX**
```postgresql
CREATE VIEW view_name AS
query
WITH [LOCAL|CASCADED] CHECK OPTION;
```

When you create a view `WITH CHECK OPTION`, PostgreSQL will ensure that you can only modify data of the view that satisfies the condition in the view’s defining query (**query**).

**Scope of check**

In PostgreSQL, you can specify a scope of check:
* `LOCAL`
* `CASCADED`

The `LOCAL` scope restricts the check option enforcement to the current view only. It does not enforce the check on the views that the current view is based on.

The `CASCADED` scope extends the check option enforcement to all underlying views of the current view. `CASCADED` is the default behavior. 

> Use the `WITH CHECK OPTION` clause to enforce constraints on data modifications through views and ensure that only valid data can be changed.

# Materialized Views

**Materialized views** store the result of a query physically and refresh the data from base tables periodically.
* Materialized views cache the result set of an expensive query and allow you to refresh data periodically.
* The materialized views can be useful in many cases that require fast data access.
* Therefore, you often find them in data warehouses and business intelligence applications.

**Creating materialized views**
```postgresql
CREATE MATERIALIZED VIEW [IF NOT EXISTS] view_name
AS
query
WITH [NO] DATA;
```
* If you want to load data into the materialized view at creation time, use the `WITH DATA` option; otherwise, use the `WITH NO DATA` option.
* If you use the `WITH NO DATA` option, the view is flagged as unreadable. It means that you cannot query data from the view until you load data into it.

If you attempt to do so, you’ll get the following error message:
```postgresql
[Err] ERROR: materialized view "view_name" has not been populated
HINT: Use the REFRESH MATERIALIZED VIEW command.
```

**Refreshing data for materialized views**

To load data into a materialized view, you use the `REFRESH MATERIALIZED VIEW` statement:
```postgresql
REFRESH MATERIALIZED VIEW view_name;
```
* When you refresh data for a materialized view, PostgreSQL locks the underlying tables.
* Consequently, you will not be able to retrieve data from underlying tables while data is loading into the view.

To avoid this, you can use the `CONCURRENTLY` option.
```postgresql
REFRESH MATERIALIZED VIEW CONCURRENTLY view_name;
```
With the `CONCURRENTLY` option, PostgreSQL creates a temporary updated version of the materialized view, compares two versions, and performs `INSERT` and `UPDATE` only the differences.
* PostgreSQL allows you to retrieve data from a materialized view while it is being updated.
* One requirement for using the `CONCURRENTLY` option is that the materialized view must have a `UNIQUE` index.
> Notice that the `CONCURRENTLY` option is only available in PostgreSQL 9.4 or later.

**Removing materialized views**
```postgresql
DROP MATERIALIZED VIEW view_name;
```

# Recursive View

In PostgreSQL, a recursive view is a view whose defining query references the view name itself.

A recursive view can be useful in performing hierarchical or recursive queries on hierarchical data structures stored in the database.

**SYNTAX**
```postgresql
CREATE RECURSIVE VIEW view_name(columns)
AS
query;
```
* The `CREATE RECURSIVE VIEW` statement is syntax sugar for a standard recursive query.
* You can add an optional schema to the name of the view.

The `CREATE RECURSIVE VIEW` statement is equivalent to the following statement:
```postgresql
CREATE VIEW view_name
AS
  WITH RECURSIVE cte_name (columns) AS (
    SELECT ...)
  SELECT columns FROM cte_name;
```

# ALTER VIEW statement

The `ALTER VIEW` statement allows you to change various properties of a view.

If you want to change the view’s defining query, use the `CREATE OR REPLACE VIEW` statement.

Here’s the basic syntax of the `ALTER VIEW` statement:
```postgresql
ALTER VIEW [IF EXISTS] view_name
action;
```

**Renaming a view**
```postgresql
ALTER VIEW [ IF EXISTS ] view_name
RENAME TO new_view_name;
```

**Changing the view option**
```postgresql
ALTER VIEW [ IF EXISTS ] view_name
SET ( view_option_name [= view_option_value] [, ... ] );
```

The `view_option_name` can be:
* `check_option`: change the check option. The valid value is `local` or `cascaded`.
* `security_barrier`: change the security-barrier property of a view. The valid value is `true` or `false`.
* `security_invoker`: change the security invoker of a view. The valid value is `true` or `false`.

**EXAMPLE**:
```postgresql
ALTER VIEW film_rating
SET (check_option = local);
```

**Changing the view column**
```postgresql
ALTER VIEW [ IF EXISTS ] view_name
RENAME [ COLUMN ] column_name TO new_column_name;
```

**Setting the new schema**
```postgresql
ALTER VIEW [ IF EXISTS ] view_name
SET SCHEMA new_schema;
```

# Listing view using SQL statement

PostgreSQL offers various database views that contain information about objects defined in the current database through the `information_schema`.

To retrieve the information about database views, you can execute the following SQL statement:
```postgresql
SELECT
  table_schema,
  table_name
FROM
  information_schema.views
WHERE
  table_schema NOT IN (
    'information_schema', 'pg_catalog'
  )
ORDER BY
  table_schema,
  table_name;
``

# Listing materialized views

To retrieve all materialized views, you can query them from the `pg_matviews` view:
```postgresql
SELECT * FROM pg_matviews;
```

The output includes detailed information about materialized views, including their definitions.